In [ ]:
import pandas as pd
import numpy as np

# ===============================
# CONFIGURAÇÕES
# ===============================
date_start = "2025-01-01"
date_end   = "2025-12-01"

lancamentos = r"C:\Users\dorag\Downloads\Lancamentos_CGL_2025.xlsx"
# CLIENTE_ALVO = "UNIVERSO CABELOS E COSMETICOS LTDA"
# CLIENTE_ALVO = "VIA CABOS SERVIÇOS DE TELECOMUNICAÇÕES LTDA"

# ===============================
# LEITURA
# ===============================
df = pd.read_excel(lancamentos, engine="openpyxl")

# ===============================
# LIMPEZA E SELEÇÃO
# ===============================
df = df[["Data", "Conta Débito", "Conta Crédito", "Valor", "Descrição Histórico"]]
df["Data"] = pd.to_datetime(df["Data"])

df = df[
    (df["Data"] >= date_start) &
    (df["Data"] < date_end)
]

# ===============================
# IDENTIFICAR CONTAS CLIENTE (70)
# ===============================
df["temp_debito"] = df["Conta Débito"].str.extract(r"^(\d{2})")
df["temp_credito"] = df["Conta Crédito"].str.extract(r"^(\d{2})")

df = df[
    (df["temp_debito"] == "70") |
    (df["temp_credito"] == "70")
]

# ===============================
# CRIAR COLUNA CLIENTE
# ===============================
df["Cliente"] = np.where(
    df["temp_credito"] == "70",
    df["Conta Crédito"].str.extract(r"-\s*(.*)")[0],
    np.where(
        df["temp_debito"] == "70",
        df["Conta Débito"].str.extract(r"-\s*(.*)")[0],
        pd.NA
    )
)

df["Cliente"] = df["Cliente"].str.strip().str.upper()

# ===============================
# FILTRAR CLIENTE DE INTERESSE
# ===============================
# df = df[df["Cliente"] == CLIENTE_ALVO].copy()

# ===============================
# PARTIDA DOBRADA (NORMALIZAÇÃO)
# ===============================
df_debito = df[["Data", "Conta Débito", "Valor", "Cliente", "Descrição Histórico"]].copy()
df_debito["Conta Completa"] = df_debito["Conta Débito"]
df_debito["D/C"] = "D"
df_debito["Valor"] = -df_debito["Valor"]

df_credito = df[["Data", "Conta Crédito", "Valor", "Cliente", "Descrição Histórico"]].copy()
df_credito["Conta Completa"] = df_credito["Conta Crédito"]
df_credito["D/C"] = "C"

df_final = pd.concat(
    [
        df_debito[["Data", "Conta Completa", "D/C", "Valor", "Cliente", "Descrição Histórico"]],
        df_credito[["Data", "Conta Completa", "D/C", "Valor", "Cliente", "Descrição Histórico"]],
    ],
    ignore_index=True
)

# ===============================
# QUEBRAR CONTA
# ===============================
df_final["Conta Código"] = df_final["Conta Completa"].str.extract(r"^(\d+)")
df_final["Conta Nome"] = df_final["Conta Completa"].str.extract(r"-\s*(.*)")
df_final["valor_abs"] = df_final["Valor"].abs().round(2)

# ===============================
# CLASSIFICAÇÃO DE CONTA
# ===============================
def classificar_conta(codigo):
    codigo = str(codigo)
    if codigo.startswith("70"):
        return "CLIENTE"
    elif codigo.startswith("16"):
        return "RECEITA"
    elif codigo.startswith(("10", "11")):
        return "FINANCEIRO"
    else:
        return "OUTRO"

df_final["tipo_conta"] = df_final["Conta Código"].apply(classificar_conta)

# ===============================
# CONCILIAÇÃO DIRECIONADA (CORE)
# ===============================
df_final = df_final.sort_values("Data").reset_index(drop=True)

df_final["status_conciliacao"] = "NAO CONCILIADO"
df_final["id_conciliacao"] = pd.NA

conciliacao_id = 1

for idx, row in df_final.iterrows():

    if df_final.at[idx, "status_conciliacao"] == "CONCILIADO":
        continue

    valor = row["valor_abs"]
    cliente = row["Cliente"]
    data = row["Data"]
    tipo = row["tipo_conta"]
    dc = row["D/C"]

    # ===============================
    # C = RECEITA  → procura D = BANCO/CAIXA
    # ===============================
    if tipo == "RECEITA" and dc == "C":
        candidatos = df_final[
            (df_final.index != idx) &
            (df_final["tipo_conta"] == "FINANCEIRO") &
            (df_final["D/C"] == "D") &
            (df_final["valor_abs"] == valor) &
            (df_final["Cliente"] == cliente) &
            (df_final["Data"] >= data) &
            (df_final["status_conciliacao"] == "NAO CONCILIADO")
        ]

    # ===============================
    # D = CLIENTE → procura C = CLIENTE
    # ===============================
    elif tipo == "CLIENTE" and dc == "D":
        candidatos = df_final[
            (df_final.index != idx) &
            (df_final["tipo_conta"] == "CLIENTE") &
            (df_final["D/C"] == "C") &
            (df_final["valor_abs"] == valor) &
            (df_final["Cliente"] == cliente) &
            (df_final["Data"] >= data) &
            (df_final["status_conciliacao"] == "NAO CONCILIADO")
        ]

    else:
        continue

    # ===============================
    # SE ENCONTRAR PAR
    # ===============================
    if not candidatos.empty:
        idx_par = candidatos.index[0]

        df_final.at[idx, "status_conciliacao"] = "CONCILIADO"
        df_final.at[idx_par, "status_conciliacao"] = "CONCILIADO"

        df_final.at[idx, "id_conciliacao"] = conciliacao_id
        df_final.at[idx_par, "id_conciliacao"] = conciliacao_id

        conciliacao_id += 1

# ===============================
# CLASSIFICAÇÃO FINAL DE STATUS
# ===============================

def definir_status(row):
    # Se foi conciliado no matching, acabou
    if row["status_conciliacao"] == "CONCILIADO":
        return "CONCILIADO"

    tipo = row["tipo_conta"]
    dc = row["D/C"]

    # VENDA EXISTE, MAS NÃO RECEBEU
    if tipo == "RECEITA" and dc == "C":
        return "NF EM ABERTO"

    if tipo == "CLIENTE" and dc == "D":
        return "NF EM ABERTO"

    # RECEBEU, MAS NÃO TEM NF
    if tipo == "CLIENTE" and dc == "C":
        return "RECEBIDO SEM NF"

    if tipo == "FINANCEIRO" and dc == "D":
        return "RECEBIDO SEM NF"

    # RECEITA SEM CLIENTE (caso defensivo)
    if tipo == "RECEITA" and dc == "C" and pd.isna(row["Cliente"]):
        return "RECEITA SEM CLIENTE"

    return "OUTRO"


df_final["status_conciliacao"] = df_final.apply(definir_status, axis=1)

# ===============================
# RESULTADO FINAL
# ===============================
df_final = df_final[
    ["Data",  "Cliente", "Conta Código", "Conta Nome", "D/C", 'tipo_conta', 'status_conciliacao', "Valor", 'Descrição Histórico']
]

df_final
